In [ ]:
!pip install transformers datasets -q

In [ ]:
deps = [
    "pandas",
    "numpy",
    "torch",
    "scikit-learn",
    "transformers",
    "datasets"
]

import pkg_resources
with open("/content/drive/MyDrive/Datathon 2025 (HidupJ0kow1)/requirements_1.txt", "w") as f:
    for pkg in deps:
        try:
            v = pkg_resources.get_distribution(pkg).version
            f.write(f"{pkg}=={v}\n")
        except:
            pass

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_dataset
import torch.serialization
from torch.serialization import add_safe_globals

In [ ]:
test_args = TrainingArguments(output_dir="./test")
print(test_args.output_dir)

./test


In [ ]:
import transformers
print(transformers.__version__)

4.53.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/datathon_test/train_df.csv')

In [ ]:
required_columns = ['attack_cat_enc', 'dur', 'dsport', 'sbytes', 'spkts', 'pkt_ratio', 'sjit', 'sload']

missing_columns = [col for col in required_columns if col not in data.columns]

if missing_columns:
    print("Kolom berikut TIDAK ditemukan dalam dataset:", missing_columns)
else:
    print("Semua kolom yang dibutuhkan ADA dalam dataset.")

Semua kolom yang dibutuhkan ADA dalam dataset.


In [ ]:
df_normal = data[data['attack_cat_enc'] == 5].copy()
df_attack = data[data['attack_cat_enc'] != 5].copy()

In [ ]:
def explain_normal(row):
    return f"Trafik normal terdeteksi dengan durasi {row['dur']} detik, data keluar masuk sebesar {row['sbytes']}B/{row['dbytes']}B, dan port sumber {row['sport']}."

In [ ]:
attack_mapping = {
    0: 'analysis', 1: 'backdoors', 2: 'dos exploits', 3: 'fuzzers',
    4: 'generic', 6: 'reconnaissance', 7: 'shellcode', 8: 'worms'
}

def explain_attack(row):
    attack_type = attack_mapping.get(row['attack_cat_enc'], 'unknown')
    return f"Serangan {attack_type} terdeteksi dengan durasi {row['dur']} detik, dari port sumber {row['sport']} ke port tujuan {row['dsport']}, ukuran data {row['sbytes']}B, jumlah paket {row['spkts']}, dan rasio paket {row['pkt_ratio']:.2f}."

In [ ]:
def generate_xai_text(row):
    if row['attack_cat_enc'] == 5:
        if row['dur'] < 0.05 and row['sbytes'] < 1000:
            return "Trafik normal berdurasi pendek dengan jumlah byte kecil."
        elif row['dur'] > 5 and row['sbytes'] > 10000:
            return "Trafik normal berdurasi panjang dengan jumlah byte cukup besar."
        else:
            return "Trafik normal dengan karakteristik umum."
    else:
        if row['dur'] > 5 and row['sbytes'] > 10000:
            return "Durasi panjang dan jumlah byte besar mengindikasikan potensi serangan."
        elif row['dbytes'] > 10000 and row['pkt_ratio'] > 2:
            return "Perbandingan paket yang tidak seimbang dan byte besar mengindikasikan serangan."
        elif row['jit_ratio'] > 5:
            return "Variasi jeda paket tinggi terdeteksi, indikasi lalu lintas serangan."
        else:
            return "Trafik mencurigakan dengan indikasi serangan ringan."

In [ ]:
df_normal['xai_template'] = df_normal.apply(explain_normal, axis=1)
df_normal['xai_logic'] = df_normal.apply(generate_xai_text, axis=1)

df_attack['xai_template'] = df_attack.apply(explain_attack, axis=1)
df_attack['xai_logic'] = df_attack.apply(generate_xai_text, axis=1)

In [ ]:
df_attack.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sload,dload,spkts,...,service_src_ratio,service_dst_ratio,res_bdy_rate,stime_hour,stime_weekday,ltime_hour,ltime_weekday,attack_cat_enc,xai_template,xai_logic
3,47439,53,0.000009,114,0,254,0,5.066666e+07,0.00,2,...,16.499918,16.499918,0.0,9,2,9,2,4,Serangan generic terdeteksi dengan durasi 9e-0...,Trafik mencurigakan dengan indikasi serangan r...
7,39299,58278,0.815515,364,1248,62,252,3.129311e+03,10712.25,8,...,0.999990,0.999990,0.0,12,3,12,3,2,Serangan dos exploits terdeteksi dengan durasi...,"Variasi jeda paket tinggi terdeteksi, indikasi..."
9,47439,53,0.000001,114,0,254,0,4.560000e+08,0.00,2,...,6.999965,6.999965,0.0,1,2,1,2,4,Serangan generic terdeteksi dengan durasi 1e-0...,Trafik mencurigakan dengan indikasi serangan r...
34,54965,111,0.000008,168,0,254,0,8.400000e+07,0.00,2,...,1.999990,0.499998,0.0,11,2,11,2,6,Serangan reconnaissance terdeteksi dengan dura...,Trafik mencurigakan dengan indikasi serangan r...
37,1043,53,0.000003,114,0,254,0,1.520000e+08,0.00,2,...,21.999890,17.999910,0.0,3,2,3,2,4,Serangan generic terdeteksi dengan durasi 3e-0...,Trafik mencurigakan dengan indikasi serangan r...


In [ ]:
df_normal['target_text'] = df_normal['xai_template'] + ". Penjelasan tambahan: " + df_normal['xai_logic']

df_attack['target_text'] = df_attack['xai_template'] + ". Penjelasan tambahan: " + df_attack['xai_logic']

In [ ]:
df_normal.to_csv('/content/drive/MyDrive/datathon_test/df_normal.csv', index=False)
df_attack.to_csv('/content/drive/MyDrive/datathon_test/df_attack.csv', index=False)

In [ ]:
df_normal = pd.read_csv('/content/drive/MyDrive/datathon_test/df_normal.csv')
df_attack = pd.read_csv('/content/drive/MyDrive/datathon_test/df_attack.csv')

In [ ]:
X_normal = df_normal.drop(columns=['target_text', 'attack_cat_enc', 'xai_template', 'xai_logic'])
y_normal = df_normal['target_text']

X_attack = df_attack.drop(columns=['target_text', 'attack_cat_enc', 'xai_template', 'xai_logic'])
y_attack = df_attack['target_text']

In [ ]:
scaler_normal = StandardScaler()
X_normal_scaled = scaler_normal.fit_transform(X_normal)

scaler_attack = StandardScaler()
X_attack_scaled = scaler_attack.fit_transform(X_attack)

tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
input_normal = [f"fitur: {','.join(map(str, row))}" for row in X_normal_scaled]
input_attack = [f"fitur: {','.join(map(str, row))}" for row in X_attack_scaled]

In [ ]:
from sklearn.utils import resample
input_normal, y_normal = resample(input_normal, y_normal, n_samples=1000000, random_state=42)

In [ ]:
def tokenize_in_batches(input_texts, label_texts, tokenizer, batch_size=10000, max_len=128):
    input_batches = []
    label_batches = []

    total_batches = (len(input_texts) + batch_size - 1) // batch_size

    for i in range(0, len(input_texts), batch_size):
        batch_idx = i // batch_size + 1
        print(f"Tokenizing batch {batch_idx}/{total_batches} ...")

        input_batch = input_texts[i:i + batch_size]
        label_batch = label_texts[i:i + batch_size]

        tokenized_inputs = tokenizer(
            input_batch,
            padding="max_length",
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )

        tokenized_labels = tokenizer(
            label_batch,
            padding="max_length",
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )["input_ids"]

        input_batches.append(tokenized_inputs)
        label_batches.append(tokenized_labels)

    # Gabungkan semua batch jadi satu tensor besar
    input_ids = {
        key: torch.cat([batch[key] for batch in input_batches], dim=0)
        for key in input_batches[0]
    }
    label_ids = torch.cat(label_batches, dim=0)

    print("Tokenization selesai.")
    return input_ids, label_ids

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

tokenized_normal, labels_normal = tokenize_in_batches(
    input_normal,
    y_normal.tolist(),
    tokenizer,
    batch_size=10000
)

tokenized_attack, labels_attack = tokenize_in_batches(
    input_attack,
    y_attack.tolist(),
    tokenizer,
    batch_size=10000
)

🔄 Tokenizing batch 1/100 ...
🔄 Tokenizing batch 2/100 ...
🔄 Tokenizing batch 3/100 ...
🔄 Tokenizing batch 4/100 ...
🔄 Tokenizing batch 5/100 ...
🔄 Tokenizing batch 6/100 ...
🔄 Tokenizing batch 7/100 ...
🔄 Tokenizing batch 8/100 ...
🔄 Tokenizing batch 9/100 ...
🔄 Tokenizing batch 10/100 ...
🔄 Tokenizing batch 11/100 ...
🔄 Tokenizing batch 12/100 ...
🔄 Tokenizing batch 13/100 ...
🔄 Tokenizing batch 14/100 ...
🔄 Tokenizing batch 15/100 ...
🔄 Tokenizing batch 16/100 ...
🔄 Tokenizing batch 17/100 ...
🔄 Tokenizing batch 18/100 ...
🔄 Tokenizing batch 19/100 ...
🔄 Tokenizing batch 20/100 ...
🔄 Tokenizing batch 21/100 ...
🔄 Tokenizing batch 22/100 ...
🔄 Tokenizing batch 23/100 ...
🔄 Tokenizing batch 24/100 ...
🔄 Tokenizing batch 25/100 ...
🔄 Tokenizing batch 26/100 ...
🔄 Tokenizing batch 27/100 ...
🔄 Tokenizing batch 28/100 ...
🔄 Tokenizing batch 29/100 ...
🔄 Tokenizing batch 30/100 ...
🔄 Tokenizing batch 31/100 ...
🔄 Tokenizing batch 32/100 ...
🔄 Tokenizing batch 33/100 ...
🔄 Tokenizing batch 

In [ ]:
class TextGenDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

In [ ]:
import random

subset_size = 200_000

indices = random.sample(range(len(labels_normal)), subset_size)
tokenized_normal_subset = {k: [v[i] for i in indices] for k, v in tokenized_normal.items()}
labels_normal_subset = [labels_normal[i] for i in indices]

dataset_normal = TextGenDataset(tokenized_normal_subset, labels_normal_subset)

In [ ]:
dataset_normal = TextGenDataset(tokenized_normal, labels_normal)
dataset_attack = TextGenDataset(tokenized_attack, labels_attack)

In [ ]:
torch.save(dataset_normal, '/content/drive/MyDrive/datathon_test/dataset_normal_resamp.pt')
torch.save(dataset_attack, '/content/drive/MyDrive/datathon_test/dataset_attack.pt')

In [ ]:
add_safe_globals([TextGenDataset])

dataset_normal = torch.load("/content/drive/MyDrive/datathon_test/dataset_normal_resamp.pt", weights_only=False)
dataset_attack = torch.load("/content/drive/MyDrive/datathon_test/dataset_attack.pt", weights_only=False)

In [ ]:
model_normal = T5ForConditionalGeneration.from_pretrained("t5-small")
model_attack = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,                          # Fokus batasan step
    per_device_train_batch_size=192,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    save_strategy="steps",
    # evaluation_strategy="no",
    fp16=True,
    dataloader_num_workers=2,
    report_to="tensorboard"
)

trainer_normal = Trainer(
    model=model_normal,
    args=training_args,
    train_dataset=dataset_normal
)

trainer_attack = Trainer(
    model=model_attack,
    args=training_args,
    train_dataset=dataset_attack
)

trainer_normal.train()
trainer_attack.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,2.376300
1000,0.319400
1500,0.283300
2000,0.261400
2500,0.243900
3000,0.231700
3500,0.222400
4000,0.214200
4500,0.208500
5000,0.203900


TrainOutput(global_step=10420, training_loss=0.31787381309465346, metrics={'train_runtime': 2289.9315, 'train_samples_per_second': 873.389, 'train_steps_per_second': 4.55, 'total_flos': 6.7670900736e+16, 'train_loss': 0.31787381309465346, 'epoch': 10.0})

In [ ]:
from transformers import T5Tokenizer

model_normal.save_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1")
model_attack.save_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-attack-explainer-v1")

tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenizer.save_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1")
tokenizer.save_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-attack-explainer-v1")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


('/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1/tokenizer_config.json',
 '/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1/special_tokens_map.json',
 '/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1/spiece.model',
 '/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/datathon_test/saved_model/t5-normal-explainer-v1")

model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
dummy_input = "fitur: 0.1,0.2,0.3,0.4,0.5"
input_ids = tokenizer.encode(dummy_input, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=50)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Trafik normal terdeteksi dengan durasi 0.4, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.


In [ ]:
def vector_to_prompt(vec):
    return "fitur: " + ",".join(f"{x:.10f}" for x in vec)

test_vectors = [
    [1.3100519691, -0.5522472851, -0.0294114342, -0.3108599332, -0.2576223264,
     0.9559007926, 8.5694000790, -0.1012008205, -0.6827390346, -0.4971613772,
     0.6366064686, -0.5344575700, 1.6566076941, 0.2286318827],  # Sample 1

    [0.2851886442, -0.3681769346, -0.0268378593, -0.2612596601, -0.2483472394,
     -0.1363363572, -0.0915017137, -0.1009390248, -0.6743034715, -0.3486130093,
     0.6366064686, -0.5536035882, 0.9660000000, 0.1800000000]  # Sample 2
]

for idx, vec in enumerate(test_vectors):
    prompt = vector_to_prompt(vec)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    output_ids = model.generate(
        input_ids,
        max_length=500,  # Lebih panjang
        num_beams=2,     # Beam search
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"🧪 Input #{idx+1}:\n{prompt}\n🧠 Output:\n{output}\n{'-'*50}")

🧪 Input #1:
fitur: 1.3100519691,-0.5522472851,-0.0294114342,-0.3108599332,-0.2576223264,0.9559007926,8.5694000790,-0.1012008205,-0.6827390346,-0.4971613772,0.6366064686,-0.5344575700,1.6566076941,0.2286318827
🧠 Output:
Trafik normal terdeteksi dengan durasi 0.292828 detik, data keluar masuk sebesar 258.0B/0.0B, dan port sumber 58632.0.. Penjelasan tambahan: Trafic normal denkan karakteristik umum.
--------------------------------------------------
🧪 Input #2:
fitur: 0.2851886442,-0.3681769346,-0.0268378593,-0.2612596601,-0.2483472394,-0.1363363572,-0.0915017137,-0.1009390248,-0.6743034715,-0.3486130093,0.6366064686,-0.5536035882,0.9660000000,0.1800000000
🧠 Output:
Trafik normal terdeteksi dengan durasi 0.263853 detik, data keluar masuk sebesar 2158.0B/2478.0b, dan port sumber 37298.0.. Penjelasan tambahan: Trafic normal denkan karakteristik umum.
--------------------------------------------------


In [ ]:
# Contoh lihat isi dataset_normal
for i in range(3):
    print("INPUT:", tokenizer.decode(dataset_normal[i]['input_ids'], skip_special_tokens=True))
    print("LABEL:", tokenizer.decode(dataset_normal[i]['labels'], skip_special_tokens=True))
    print("---")

INPUT: fitur: 0.24245988790096953,-0.7168990016151491,0.21425086662503318,-0.13645319704050213,-0.23585742395054662,-0.1363363572871558,-0.09150171377545706,-0.10149564717829447,-0.6808674435284124,0.18616111496703294,0.6366064686340912,0.2599448175094449,0.2788012776
LABEL: Trafik normal terdeteksi dengan durasi 3.026773 detik, data keluar masuk sebesar 2934.0B/3742.0B, dan port sumber 37234.0.. Penjelasan tambahan: Trafik normal dengan karakteristik umum.
---
INPUT: fitur: -0.7410850430231753,-0.7106972389078172,-0.05199232064804422,0.299921768092186,-0.1677528055296078,-0.1363363572871558,-0.09150171377545706,-0.0638428552536884,0.16437490845160563,1.2259996900081045,0.6366064686340912,-0.6963840297715552,-0.6955624758513388
LABEL: Trafik normal terdeteksi dengan durasi 0.031301 detik, data keluar masuk sebesar 7808.0B/15182.0B, dan port sumber 18405.0.. Penjelasan tambahan: Trafik normal dengan karakteristik umum.
---
INPUT: fitur: -1.0328211118112869,-0.17088971276718995,-0.051758